## Wrangling Efforts With @WeRateDogs Twitter Data

Data relating to the @WeRateDogs Twitter account was gathered from three places: 
- A .csv downloaded from Udacity containing most of the data for each tweet, stored in `df_tweets`
- A .tsv programmatically downloaded from Udacity containing machine learning prediction data for dog breeds, stored in `df_preds`
- Data was scraped from the Twitter API to gather additional data for tweets in the .csv file using the `tweepy` and `json` packages in Python, stored in `df_json`

Data from each of these sources was copied and saved, and then placed into a pandas dataframe for wrangling and analysis. In assessing the data, a numerous issues were identified as needing to be addressed so that the data could be properly analyzed.

### Data Issues Requiring Cleaning

- TIDINESS ISSUE # 1: In `df_tweets`, 'in_reply_to_status_id' & 'in_reply_to_user_id' are stored as floats rather than strings, and worse, they're stored in scientific notation making them unusable at present

**Fix:** This problem had to be fixed to allow for navigation of the data in these columns. Conversion directly to string would retain the "e-xx" scientific notation as if it were part of the value, so the data had to be converted to integer form first, to get rid of the scientific notation. The column itself could not be converted to integer because of the `NaN`s in the column, so the needed data was copied to a new dataframe, converted to integer to get the correct value, then converted to string. Once converted, this data was used to replace the values (and column type) in the original dataframe.

- QUALITY ISSUE # 1: In `df_tweets`, 'timestamp' should be converted from object to datetime, 'tweet_id' should be converted from int64 to string (this is the unique portion of the tweet's URL)

**Fix:** This was done using `astype()` for 'tweet_id', and `to_datetime()` for the 'timestamp'.

- QUALITY ISSUE # 2: In `df_tweets`, `expanded_urls` is missing values for a small portion of the tweets in the dataset

**Fix:** Valid, clickable URLs were generated for the rows missing them, by appending the row's 'tweet_id' to the prefix string 'https://twitter.com/dog_rates/status/', like so: [https://twitter.com/dog_rates/status/671550332464455680](https://twitter.com/dog_rates/status/671550332464455680).


- QUALITY ISSUE # 3: In `df_tweets`, the 'source' column is irrelevant

**Fix:** This column was dropped from the dataframe with `drop()`.


- QUALITY ISSUE # 4: In `df_tweets`, some dogs are incorrectly named "a"

**Fix:** Dog names were updated from "a" to "None" by using `replace()` on the name column of the dataframe. 


- QUALITY ISSUE # 5: In `df_tweets`, 'None' values for 'dog_stage' should be `NaN`

**Fix:** Dog stages were updated from "None" to `NaN` by using `replace("None", np.nan)` on each dog_stage column.


- TIDINESS ISSUE # 2: In `df_tweets`, 'doggo', 'floofer', 'pupper', and 'puppo' should be combined into one 'dog_stage' column

**Fix:** All four columns were concatenated into a single 'dog_stage' column with `cat()`, substituting an empty string for `NaN` values. Then, 'dog_stage' that were left empty were set to `NaN` with `replace("", np.nan)`. 12 rows were found to have multiple dog stages (e.g. 'flooferpupper'). Visual assessment was done on each of these tweets to determine the appropriate course of action. Several rows had their value corrected with `replace()` to the correct value or were deleted when they were found not to be a rating of a dog at all. A handful were found to be ratings for two dogs, one a 'doggo' and the other a 'puppo', and these rows were copied into a new dataframe. The original rows were then updated to 'puppo' with `at()`, the copied rows updated to 'doggo', and then the copied rows were appended to the bottom of the dataframe. 


- QUALITY ISSUE # 6: In `df_tweets`, filter through reply tweets by reply_to fields to remove "outdated" ratings and retain modified ratings

**Fix:** Replies to external users (not to self) were retained, as visual assessment indicated that these were generally new dog ratings in response to requests from other users. Replies to self were sampled and visual assessment found that they often consist of updates to prior ratings. A `query()` was used to find the original 'tweet_id' for each reply-to-self, and if the original tweet_id existed in the dataframe, that tweet_id (row) was deleted with `drop()` to avoid counting a since-amended rating. 


- QUALITY ISSUE # 7: In `df_tweets`, delete ratings which are not ratings of dogs, or for which the original rating is unclear if it is for the owner or the dog or is rating Snoop Dogg

**Fix:** An examination of ratings > 14 (the stated 'maximum' by @WeRateDogs) found numerous errors in the database. Some were averaged ratings (for example, a litter of 5 puppies receiving 55/50, a score of 11/10 for each of 5 dogs), some were not ratings at all, some were no ratings of dogs (one was a rating of Snoop Dogg), and some had accidentally been populated with the wrong numbers (for example, a 13/10 on 4/20 was mistakenly documented as a 4/20). Only one was legitimately a rating of a dog higher than 14. All of these issues were documented in a table in the master report. 
For the ratings that were not ratings at all, these were removed from the dataframe by finding the row with the 'tweet_id' in question and then using `drop()` to remove it from the dataframe. 


- QUALITY ISSUE # 8: In `df_tweets`, correct numerators and denominators for group/litter ratings

**Fix:** For the rows mentioned above which had issues with average/cumulative ratings, these were fixed with by finding the row with the 'tweet_id' in question and then using `at()` to fix the rating numerator and denominator, as appropriate. 


- QUALITY ISSUE # 9: In `df_tweets`, cast numerator column as float

**Fix:** Some dogs were found to have been given incorrect ratings because their ratings were actually in decimal form (for example, 9.75/10). This required that the 'rating_numerator' column be cast to float using `astype()`.


- QUALITY ISSUE # 10: In `df_tweets`, correct ratings for ratings given in decimal form

**Fix:** For the rows mentioned above which had decimals in their ratings, these were fixed with by finding the row with the 'tweet_id' in question and then using `at()` to fix the rating numerator and denominator, as appropriate. 


- QUALITY ISSUE # 11: In `df_tweets`, correct rating for Atticus, patriotic woofboy from 1776/10 to 14/10

**Fix:** As mentioned, only [one tweet](https://twitter.com/dog_rates/status/749981277374128128) had what appeared to be a genuine rating above 14/10, that being 1776/10. Subsequent tweets from @WeRateDogs indicate that the maximum is 14, so `at()` was used to amend this rating to 14/10. 


- QUALITY ISSUE # 12: In `df_tweets`, correct ratings for remaining tweets with denominator unequal to 10

**Fix:** Several tweets had denominators unequal to 10, most of which had been fixed in the previous steps. Visual assessment of these 6 tweets found that 2 were not dog ratings at all, and the other 4 were errors. The 2 that were not ratings were removed from the dataset with `drop()`, and the remaining 4 had their numerators and denominators amended as appropriate with `at()`. 

- QUALITY ISSUE # 13: In `df_json`, cast 'retweet_count' and 'favorite_count' as integers, and 'tweet_id' as string

**Fix:** 'retweet_count' and 'favorite_count' were cast as integers and 'tweet_id' was cast as string, all using `astype()`.


- TIDINESS ISSUE # 3: In `df_json`, no column exists to measure total engagement, which retweets and favorites are both examples of 

**Fix:** Engagement was going to be an important part of my analysis, so a "total engagement" column was created by adding the values in 'retweet_count' and 'favorite_count'. 


- TIDINESS ISSUE # 4: Join `df_json` to `df_tweets`

**Fix:** `df_json` consists of additional data scraped from the Twitter API about the tweets located in `df_tweets`. As this data was definitively part of the basic tweet-level data in `df_tweets`, this was joined to that dataframe. 


- QUALITY ISSUE # 14: In`df_preds`, 'tweet_id' needs to be cast as string

**Fix:** In the dataframe for the machine learning dog breed predictions, the 'tweet_id' needed to be cast as a string using `astype()`, similar to what was done in the other two dataframes.

### Outcomes

This was a laborious process, as many of the issues requiring attention were labor-intensive because they required visual assessment and individualized solutions to make sure the dataset was as accurate as possible to enable analysis. The combined data from `df_tweets` and `df_json` was placed into a dataframe named `df_master`, and the final cleaned dataframes were stored in two files, according to tidy data principles:
- `df_master` was stored in 'twitter_archive_master.csv' with UTF-8 encoding
- `df_preds` was stored in 'twitter_predictions_master.csv' with UTF-8 encoding
The data in `df_master` was then used successfully to facilitate analysis of the data to answer several questions and generate appropriate visualizations.